In [1]:
import os
import sys
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append("../gotmtool")
from gotmtool import *

## Create a model
Create a model with environment file `../../.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [2]:
casename = 'ERA5_plus_STD'
datadir = '/Users/qingli/work/saildrone_southern_ocean/'+casename
m = Model(name=casename, environ='../gotmtool/.gotm_env.yaml')

Take a look at what are defined in the environment file.

In [3]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

   gotmdir_code: /Users/qingli/work/saildrone_southern_ocean/gotm/code
   gotmdir_data: /Users/qingli/work/saildrone_southern_ocean/gotm/data
  gotmdir_build: /Users/qingli/work/saildrone_southern_ocean/gotm/build
    gotmdir_exe: /Users/qingli/work/saildrone_southern_ocean/gotm/exe
    gotmdir_run: /Users/qingli/work/saildrone_southern_ocean/gotm/run
 gotmdir_figure: /Users/qingli/work/saildrone_southern_ocean/gotm/figure
   gotmdir_tool: /Users/qingli/work/saildrone_southern_ocean/gotmtool


## Build the model

In [4]:
%%time
m.build()

CPU times: user 897 µs, sys: 6.44 ms, total: 7.34 ms
Wall time: 80.5 ms


## Configuration
Initialize the GOTM configuration

In [5]:
cfgfile = m.environ['gotmdir_run']+'/'+casename+'/gotm.yaml'
if os.path.exists(cfgfile):
    print('loading configure file...')
    cfg = yaml_load(filename=cfgfile)
else:
    cfg = m.init_config(filename=cfgfile)

loading configure file...


Update the configuration

In [6]:
# setup
title = 'Constant forcing'
nlev = 400
cfg['title'] = title
cfg['location']['name'] = 'Idealized'
cfg['location']['depth'] = 200
cfg['time']['dt']    = 60.0
cfg['grid']['nlev']  = nlev

# output
cfg['output'] = {}
cfg['output']['gotm_out'] = {}
cfg['output']['gotm_out']['use'] = True
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['k1_stop'] = nlev+1
cfg['output']['gotm_out']['k_stop'] = nlev
cfg['output']['gotm_out']['time_unit'] = 'dt'
cfg['output']['gotm_out']['time_step'] = 30
cfg['output']['gotm_out']['variables'] = [{}]
cfg['output']['gotm_out']['variables'][0]['source'] = '*'

# forcing
cfg['temperature']['method'] = 'file'
cfg['salinity']['method'] = 'file'
cfg['surface']['fluxes']['heat']['method'] = 'constant'
cfg['surface']['fluxes']['tx']['method'] = 'constant'
cfg['surface']['fluxes']['ty']['method'] = 'constant'
cfg['surface']['fluxes']['ty']['constant_value'] = 0.0
cfg['surface']['swr']['method'] = 'file'
cfg['surface']['swr']['file'] = datadir+'/swr_era5.dat'
cfg['surface']['precip']['method'] = 'constant'
cfg['waves']['stokes_drift']['us']['method'] = 'off'
cfg['waves']['stokes_drift']['vs']['method'] = 'off'

# EOS -- use linear
cfg['eq_state']['form'] = 'full-pot'

# water type (Jerlov IB)
cfg['light_extinction']['method'] = 'jerlov-ib'

In [7]:
cfg['turbulence']['turb_method'] = 'second_order'
cfg['turbulence']['tke_method'] = 'tke'
cfg['turbulence']['len_scale_method'] = 'gls'
cfg['turbulence']['scnd']['method'] =  'weak_eq_kb_eq'
cfg['turbulence']['scnd']['scnd_coeff'] =  'canuto-a'
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.25
cfg['turbulence']['generic']['gen_m'] = 1.5 
cfg['turbulence']['generic']['gen_n'] = -1.0
cfg['turbulence']['generic']['gen_p'] = 3.0 
cfg['turbulence']['generic']['cpsi1'] = 1.44
cfg['turbulence']['generic']['cpsi2'] = 1.92
cfg['turbulence']['generic']['cpsi3minus'] = -0.63
cfg['turbulence']['generic']['cpsi3plus'] = 1.0 
cfg['turbulence']['generic']['sig_kpsi'] = 1.0 
cfg['turbulence']['generic']['sig_psi'] = 1.3

In [8]:
# collect the configurations and the labels of the two runs
cfgs = []
labels = []

In [9]:
# load list of cases
with open(os.path.join(datadir, 'caselist.txt'), 'r') as f:
    lines = f.readlines()
caselist = [line.strip() for line in lines]
caselist

['C0030',
 'C0133',
 'C0236',
 'C0339',
 'C0442',
 'C0545',
 'C0648',
 'C0751',
 'C0854']

In [10]:
nstd_hf  = [-3., -2., -1., 0., 1., 2., 3.]
nstd_tau = [-3., -2., -1., 0., 1., 2., 3.]
nstd_hf_label = ['{prefix}{val:d}'.format(prefix='p' if v > 0 else 'm' if v < 0 else '',
                                          val=abs(int(v))) for v in nstd_hf]
nstd_tau_label = ['{prefix}{val:d}'.format(prefix='p' if v > 0 else 'm' if v < 0 else '',
                                           val=abs(int(v))) for v in nstd_tau]
stau = 0.18
sqlat = 0.09
sqsens = 0.11
for cname in caselist:
    configfile = os.path.join(datadir, cname, 'config.dat')
    tproffile = os.path.join(datadir, cname, 't_prof.dat')
    sproffile = os.path.join(datadir, cname, 's_prof.dat')
    with open(configfile, 'r') as f:
        lines = f.readlines()
    startdate = lines[0].strip()
    lon, lat, hf, tau, qlat, qsens = lines[1].strip().split()
    lon = float(lon)
    lat = float(lat)
    hf = float(hf)
    tau = float(tau)
    qlat = float(qlat)
    qsens = float(qsens)
    dttime = pd.to_datetime(startdate)
    dttime_stop = dttime + pd.DateOffset(days=5)
    stopdate = dttime_stop.strftime('%Y-%m-%d %H:%M:%S')
    print('{}: {} -> {}'.format(cname, startdate, stopdate))
    print('  lon: {} lat: {}'.format(lon, lat))
    print('  Qᵣ: {:6.2f}  Qˢᵣ: {:6.2f}  Qˡᵣ: {:6.2f}  τᵣ: {:5.3f}'.format(hf, qsens, qlat, tau))
    print('-'*64)
    for i, std1 in enumerate(nstd_hf):
        hf_a  = hf + std1*(qsens*sqsens+qlat*sqlat)
        plist = []
        for j, std2 in enumerate(nstd_tau):
            tau_a = tau + std2*tau*stau
            plist.append('{:6.2f}, {:5.3f}'.format(hf_a, tau_a))
            rlabel = '-'.join([cname, 'hf', nstd_hf_label[i], 'tau', nstd_tau_label[j]])
#             print(clabel)
            # set config
            cfg['location']['latitude'] = lat
            cfg['location']['longitude'] = lon
            cfg['time']['start'] = startdate
            cfg['time']['stop']  = stopdate
            cfg['turbulence']['turb_method'] = 'second_order' 
            cfg['temperature']['file'] = tproffile
            cfg['salinity']['file'] = sproffile
            cfg['surface']['fluxes']['heat']['constant_value'] = hf_a
            cfg['surface']['fluxes']['tx']['constant_value'] = tau_a
            cfgs.append(copy.deepcopy(cfg))
            labels.append(rlabel+'_GLS-C01A')
            cfg['turbulence']['turb_method'] = 'cvmix'
            cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'none'
            cfgs.append(copy.deepcopy(cfg))
            labels.append(rlabel+'_KPP-CVMix')
        print(' | '.join(plist))          
    print('\n')

C0030: 2019-02-01 00:00:00 -> 2019-02-06 00:00:00
  lon: -163.73 lat: -53.92
  Qᵣ: -59.31  Qˢᵣ:  12.12  Qˡᵣ: -21.36  τᵣ: 0.130
----------------------------------------------------------------
-57.54, 0.060 | -57.54, 0.083 | -57.54, 0.106 | -57.54, 0.130 | -57.54, 0.153 | -57.54, 0.176 | -57.54, 0.200
-58.13, 0.060 | -58.13, 0.083 | -58.13, 0.106 | -58.13, 0.130 | -58.13, 0.153 | -58.13, 0.176 | -58.13, 0.200
-58.72, 0.060 | -58.72, 0.083 | -58.72, 0.106 | -58.72, 0.130 | -58.72, 0.153 | -58.72, 0.176 | -58.72, 0.200
-59.31, 0.060 | -59.31, 0.083 | -59.31, 0.106 | -59.31, 0.130 | -59.31, 0.153 | -59.31, 0.176 | -59.31, 0.200
-59.90, 0.060 | -59.90, 0.083 | -59.90, 0.106 | -59.90, 0.130 | -59.90, 0.153 | -59.90, 0.176 | -59.90, 0.200
-60.49, 0.060 | -60.49, 0.083 | -60.49, 0.106 | -60.49, 0.130 | -60.49, 0.153 | -60.49, 0.176 | -60.49, 0.200
-61.08, 0.060 | -61.08, 0.083 | -61.08, 0.106 | -61.08, 0.130 | -61.08, 0.153 | -61.08, 0.176 | -61.08, 0.200


C0133: 2019-02-05 00:00:00 -> 2019-0

-44.33, 0.298 | -44.33, 0.415 | -44.33, 0.531 | -44.33, 0.648 | -44.33, 0.764 | -44.33, 0.881 | -44.33, 0.998
-41.87, 0.298 | -41.87, 0.415 | -41.87, 0.531 | -41.87, 0.648 | -41.87, 0.764 | -41.87, 0.881 | -41.87, 0.998
-39.42, 0.298 | -39.42, 0.415 | -39.42, 0.531 | -39.42, 0.648 | -39.42, 0.764 | -39.42, 0.881 | -39.42, 0.998
-36.96, 0.298 | -36.96, 0.415 | -36.96, 0.531 | -36.96, 0.648 | -36.96, 0.764 | -36.96, 0.881 | -36.96, 0.998
-34.51, 0.298 | -34.51, 0.415 | -34.51, 0.531 | -34.51, 0.648 | -34.51, 0.764 | -34.51, 0.881 | -34.51, 0.998




## Run the model

In [11]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=4)

CPU times: user 700 ms, sys: 119 ms, total: 819 ms
Wall time: 3min 18s
